In [2]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from PIL import Image
from google import genai
from dotenv import load_dotenv
import os
import json
import re

In [2]:
# CONEXIÓN MONGODB
load_dotenv()
conexion = os.getenv("DATABASE_URL")
client = MongoClient(conexion, server_api=ServerApi('1'))
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

NameError: name 'load_dotenv' is not defined

In [4]:
mydb = client["Cluster0"]


In [42]:
training_modules = mydb["module"]



In [43]:
tmodule1 = {
    "title": "Módulo 3. Estrategia de crecimiento digital",
    "initialDate": "August 1, 2025",
    "finalDate": "October 15, 2025",
    "session": "Sesión 2: Optimización de embudos de conversión",
    "material": "https://drive.google.com/drive/folders/1a2b3c4d5e6f7g8h9i"   
}
tmodule2 = {
    "title": "Módulo 4: Marketing de producto y validación",
    "initialDate": "October 20, 2025",
    "finalDate": "December 5, 2025",
    "session": "Sesión 1: Estrategias de validación de mercado",
    "material": "https://drive.google.com/drive/folders/abcde12345"   
}
tmodule3 = {
    "title": "Módulo 5: Preparación para inversión y pitch",
    "initialDate": "January 10, 2026",
    "finalDate": "March 15, 2026",
    "session": "Sesión 3: Cómo construir un pitch ganador",
    "material": "https://drive.google.com/drive/folders/xyz67890"   
}

In [44]:
from datetime import datetime

modules = [tmodule1, tmodule2, tmodule3]
processed_modules = []

for mod in modules:
    doc = mod.copy()
    try:
        # Convertir fechas tipo "August 1, 2025" a datetime
        initial_dt = datetime.strptime(doc["initialDate"], "%B %d, %Y")
        final_dt = datetime.strptime(doc["finalDate"], "%B %d, %Y")
        
        doc["initialDate"] = initial_dt
        doc["finalDate"] = final_dt
        
        print(f"✅ '{doc['title']}': fechas convertidas a {initial_dt} - {final_dt}")
    except Exception as e:
        print(f"❌ Error al convertir fechas para '{doc.get('title', 'N/A')}': {e}")
    
    processed_modules.append(doc)

# Insertar en MongoDB
try:
    result = training_modules.insert_many(processed_modules)
    print(f"\n🎉 {len(result.inserted_ids)} módulos insertados correctamente.")
except Exception as e:
    print(f"❌ Error al insertar módulos en MongoDB: {e}")

✅ 'Módulo 3. Estrategia de crecimiento digital': fechas convertidas a 2025-08-01 00:00:00 - 2025-10-15 00:00:00
✅ 'Módulo 4: Marketing de producto y validación': fechas convertidas a 2025-10-20 00:00:00 - 2025-12-05 00:00:00
✅ 'Módulo 5: Preparación para inversión y pitch': fechas convertidas a 2026-01-10 00:00:00 - 2026-03-15 00:00:00

🎉 3 módulos insertados correctamente.


In [25]:
# Tabla Session

collection_session = mydb["session"]


In [21]:
session1 = {
   'name': 'Alba Gonzalez', 
   "date": "March 10, 2025",
   'position': 'Chief HR Brand & Community', 
   'email': 'a.gonzalez@sesametime.com', 
   'company': 'Sesame', 
   'linkedIn': 'https://www.linkedin.com/in/alba-gonz%C3%A1lez-vilches-23627670/', 
   'studyProgram': 'Empty', 
   'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Faa931eab-8b4f-4839-94c9-b327d4115e84%2F1738075692841.jpg/size/w=2000?exp=1752919535&sig=wggxo8aZvNYcEpoZw7nUaUNYg5AnLN-vHeD1vhjE0Hs&id=1decf8b7-c83c-80e3-bf3a-fd21454e58ff&table=block'
   }


session2 = {
    "name": "Session 2: Building a Minimum Viable Product",
    "date": "March 10, 2025",
    "trainer": "Carlos Martínez",
    "position": "Product Manager",
    "company": "InnovateXYZ",
    "material": "https://drive.google.com/drive/folders/example2",
    "survey": "https://forms.gle/surveyform2",
    "module_id": "64f1a23b9cd0e3e1f10a5678"
}

session3 = {
    "name": "Session 3: Growth Hacking Techniques",
    "date": "March 10, 2025",
    "trainer": "Linda Wang",
    "position": "Growth Strategist",
    "company": "ScaleUp Pro",
    "material": "https://drive.google.com/drive/folders/example3",
    "survey": "https://forms.gle/surveyform3",
    "module_id": "64f1a23b9cd0e3e1f10a9101"
}

In [22]:
sessions = [session1, session2, session3]
processed_sessions = []

for session in sessions:
    try:
        date_obj = datetime.strptime(session["date"], "%B %d, %Y")
        session["date"] = date_obj
        print(f" '{session['name']}': fecha convertida a {date_obj}")
    except Exception as e:
        print(f" Error al convertir fecha en '{session.get('name')}': {e}")

    processed_sessions.append(session)

try:
    result = collection_session.insert_many(processed_sessions)
    print(f"\n {len(result.inserted_ids)} sesiones insertadas correctamente.")
except Exception as e:
    print(f" Error al insertar sesiones en MongoDB: {e}")

 'Alba Gonzalez': fecha convertida a 2025-03-10 00:00:00
 'Session 2: Building a Minimum Viable Product': fecha convertida a 2025-03-10 00:00:00
 'Session 3: Growth Hacking Techniques': fecha convertida a 2025-03-10 00:00:00

 3 sesiones insertadas correctamente.


In [5]:
from datetime import datetime, timezone
from PIL import Image
from google import genai
from dotenv import load_dotenv
import os
import json
import re

gemini_client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])

def generate_response(image_path):
    prompt = '''
Extract all the text from the picture and return it in a JSON format, using English keys.
Example of desired output:

Translate the field names to English. You don't need to translate the values, just the field names.
'''
    image = Image.open(image_path)
    response = gemini_client.models.generate_content(
        model="gemini-2.5-pro",
        contents=[image, prompt]
    )
    return response.text





In [17]:
def convert_to_dict(json_file):
    json_string_clean = re.sub(r'^```json\n|\n```$', '', json_file, flags=re.MULTILINE)
    return json.loads(json_string_clean)

def process_session(data, trainer_id=""):
    # Intentamos convertir la fecha si existe
    date_str = data.get("date")
    if date_str:
        try:
            # Intenta parsear la fecha, puedes ajustar el formato según lo que venga del OCR
            session_date = datetime.strptime(date_str, "%B %d, %Y").replace(tzinfo=timezone.utc)
        except Exception:
            session_date = datetime.now(timezone.utc)
    else:
        session_date = datetime.now(timezone.utc)

    session = {
        "name": data.get("studyProgram") or data.get("name", "No Session Name"),
        "date": session_date,
        "trainer_id": trainer_id,
        "material": "",
        "survey": "",
        
        "description": "",
        "photo": data.get("photo", "")
    }
    return session

In [ ]:
# 📂 Carpeta de imágenes
folder_path = 'images/session/Captura de pantalla 2025-07-19 081827'

sessions_to_upload = []

for img_file in :
    image_path = os.path.join(folder_path, img_file)
    print(f"\n⏳ Procesando: {image_path}")
    try:
        gemini_output = generate_response(image_path)
        data_dict = convert_to_dict(gemini_output)
        session_doc = process_session(data_dict, trainer_id="")  
        sessions_to_upload.append(session_doc)
        print(f"✅ Extraído sesión: {data_dict}")
    except Exception as e:
        print(f"❌ Error en {img_file}: {e}")




⏳ Procesando: images/session/Captura de pantalla 2025-07-19 081609.png\i
❌ Error en i: [Errno 2] No such file or directory: 'images/session/Captura de pantalla 2025-07-19 081609.png\\i'

⏳ Procesando: images/session/Captura de pantalla 2025-07-19 081609.png\m
❌ Error en m: [Errno 2] No such file or directory: 'images/session/Captura de pantalla 2025-07-19 081609.png\\m'

⏳ Procesando: images/session/Captura de pantalla 2025-07-19 081609.png\a
❌ Error en a: [Errno 2] No such file or directory: 'images/session/Captura de pantalla 2025-07-19 081609.png\\a'

⏳ Procesando: images/session/Captura de pantalla 2025-07-19 081609.png\g
❌ Error en g: [Errno 2] No such file or directory: 'images/session/Captura de pantalla 2025-07-19 081609.png\\g'

⏳ Procesando: images/session/Captura de pantalla 2025-07-19 081609.png\e
❌ Error en e: [Errno 2] No such file or directory: 'images/session/Captura de pantalla 2025-07-19 081609.png\\e'

⏳ Procesando: images/session/Captura de pantalla 2025-07-19 0816

In [24]:
from datetime import datetime, timezone
from PIL import Image
from google import genai
import os
import json
import re
from bson import ObjectId

gemini_client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])

def generate_response(image_path):
    prompt = '''
Extract all the text from the picture and return it in a JSON format, using English keys.
Example of desired output:

Translate the field names to English. You don't need to translate the values, just the field names.
Always output a single JSON object, even if multiple sections appear in the image.

'''
    image = Image.open(image_path)
    response = gemini_client.models.generate_content(
        model="gemini-2.5-pro",
        contents=[image, prompt]
    )
    return response.text

def convert_to_dict(json_file):
    json_string_clean = re.sub(r'^```json\n|\n```$', '', json_file.strip(), flags=re.MULTILINE)
    try:
        return json.loads(json_string_clean)
    except json.JSONDecodeError as e:
        print(f"ERROR parsing JSON: {e}")
        print(f"JSON content was:\n{json_string_clean}")
        return {}

def parse_date(date_str):
    formats = ["%B %d, %Y", "%Y-%m-%d", "%d/%m/%Y"]
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt).replace(tzinfo=timezone.utc)
        except:
            pass
    print(f"Warning: couldn't parse date '{date_str}', using now.")
    return datetime.now(timezone.utc)

def process_session(data, trainer_ids=None):
    if trainer_ids is None:
        trainer_ids = []

    date_str = data.get("Date") or data.get("date") or data.get("sessionDate") or ""
    session_date = parse_date(date_str) if date_str else datetime.now(timezone.utc)

    name = data.get("Title") or data.get("title") or data.get("name") or "No Session Name"

    material = data.get("Material", "")
    survey = data.get("Survey", "")
    
    description = data.get("Description", "") or data.get("Content", "")
    if isinstance(description, list):
        description = "\n".join(description)

    # Convertir cada ID en trainer_ids a ObjectId si es string y no vacío
    trainer_object_ids = []
    for tid in trainer_ids:
        try:
            trainer_object_ids.append(ObjectId(tid))
        except Exception:
            print(f"⚠️ Invalid trainer ID skipped: {tid}")

    session = {
        "name": name,
        "date": session_date,
        "trainer_ids": trainer_object_ids,  # <-- lista de ObjectId
        "material": material,
        "survey": survey,
        "description": description,
    }
    return session

# --- Script para procesar carpeta ---
folder_path = 'images/session/'
sessions_to_upload = []

# Ejemplo: lista de IDs de formadores para esta sesión
trainer_ids_list = [
    "687a2dfc2b40db0b206ba843",
    "64f843d1b23edcb31f2e4f77",
    # puedes añadir más IDs
]

for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(folder_path, filename)
        print(f"\n⏳ Procesando: {image_path}")
        try:
            gemini_output = generate_response(image_path)
            data_dict = convert_to_dict(gemini_output)
            session_doc = process_session(data_dict, trainer_ids=trainer_ids_list)
            sessions_to_upload.append(session_doc)
            print(f"✅ Extraído sesión: {session_doc['name']}, fecha: {session_doc['date']}, formadores: {len(session_doc['trainer_ids'])}")
        except Exception as e:
            print(f"❌ Error en {filename}: {e}")

print(f"\n🎯 Total sesiones extraídas: {len(sessions_to_upload)}")



⏳ Procesando: images/session/Captura de pantalla 2025-07-19 081404.png
✅ Extraído sesión: Presentación del Programa. Dinámica Icebreaker Pitches y Networking Beers, fecha: 2025-02-24 00:00:00+00:00, formadores: 2

⏳ Procesando: images/session/Captura de pantalla 2025-07-19 081623.png
✅ Extraído sesión: Sesión 11. Autogestión de Liderazgo, fecha: 2025-07-19 16:19:45.560961+00:00, formadores: 2

⏳ Procesando: images/session/Captura de pantalla 2025-07-19 081626 - copia.png
✅ Extraído sesión: Sesión 12. Gestión de Personas, fecha: 2025-06-09 00:00:00+00:00, formadores: 2

⏳ Procesando: images/session/Captura de pantalla 2025-07-19 081827.png
ERROR parsing JSON: Expecting value: line 1 column 1 (char 0)
JSON content was:
I need to extract all the text from the provided image and format it as a single JSON object.
The user wants me to translate the field names (the labels) from Spanish to English, but keep the values in their original language.

Here's the plan:
1.  **Identify all the key-

In [23]:
print(sessions_to_upload)

[{'name': 'Sesión 13. Alianzas Corporativas', 'date': datetime.datetime(2025, 7, 19, 10, 43, 7, 86561, tzinfo=datetime.timezone.utc), 'trainer_id': '687a2dfc2b40db0b206ba843', 'material': 'Empty', 'survey': 'https://docs.google.com/forms/d/e/1FAIpQLSdTeGldxev0uGvw7MAquMDfhfseFcMBUXWCPvBqjsVi9xR1uA/viewform?usp=sharing', 'module_id': '687a902eec6e65fc3d202c41', 'description': 'Claves de la innovación corporativa: ¿cómo innovan las corporaciones en 2025?\nInsights y aprendizajes después de haber entrevistado a +100 corporaciones\nCasos de innovación abierta\nCómo colaborar de forma efectiva con corporaciones (procesos, templates y herramientas)\nDinámica: "Diseñar una propuesta de alianza corporativa"'}]


In [27]:

from datetime import datetime
from bson import ObjectId

session = {
    "name": "Sesión 14. Momentums Estratégico para Levantar Ronda de Inversión",
    "date": datetime(2025, 6, 23, 0, 0, 0),  # fecha tipo datetime
    "trainer_ids": [
        ObjectId("687bb87795aa2580f6dec87b"),
        ObjectId("687bb87795aa2580f6dec879"),
        ObjectId("687bb87795aa2580f6dec86b")
    ],
    "material": "",
    "survey": "https://docs.google.com/forms/d/e/1FAIpQLSf7hyvfbGwhfI0tFQlWxcM1BPLqSGo48Sc4YXNEH7eMYrllhg/viewform?usp=sharing",
    "description": "- Cuándo y cómo preparar una ronda de inversión.Métricas clave que los inversores buscan.Estrategias para generar interés y negociar términos favorables.Ejemplo de dinámica: Simulación de pitch ante inversores"
}

if session:
    result = collection_session.insert_one(session)
    print(f"\n🎉 Sesión insertada correctamente con ID: {result.inserted_id}")
else:
    print("\n⚠️ No se encontraron datos válidos para insertar.")


🎉 Sesión insertada correctamente con ID: 687bcd1e95aa2580f6dec8a7


In [8]:
user = mydb["user"]

In [9]:
user1 = {
    "firstName": "Laura",
    "lastName": "Gómez",
    "secondLastName": "Martínez",
    "email": "laura.gomez@example.com",
    "phone": "+34 600 123 456",
    "role": "mentor",
    "confirmed": True,
    "company": "TechBoost",
    "password": "hashedpassword123",
    "token": [ ]
}

user2 = {
    "firstName": "Michael",
    "lastName": "Smith",
    "secondLastName": "Brown",
    "email": "michael.smith@example.com",
    "phone": "+1 415 555 7890",
    "role": "startup",
    "confirmed": False,
    "company": "GreenStartups",
    "password": "hashedpassword456",
    "token": []
}

user3 = {
    "firstName": "Sofía",
    "lastName": "Pérez",
    "secondLastName": "López",
    "email": "sofia.perez@example.com",
    "phone": "+52 55 1234 5678",
    "role": "trainer",
    "confirmed": True,
    "company": "Innovative Minds",
    "password": "hashedpassword789",
    "token": []
}

In [10]:
resultado = user.insert_many([user1,user2,user3])
print(f"Usuario insertado con ID: {resultado.inserted_ids}")

Usuario insertado con ID: [ObjectId('687a1e6b2b40db0b206ba81e'), ObjectId('687a1e6b2b40db0b206ba81f'), ObjectId('687a1e6b2b40db0b206ba820')]


In [13]:
trainer = mydb["trainer"]

In [14]:

trainer1 = {
    'name': 'Alba Gonzalez', 
   'position': 'Chief HR Brand & Community', 
   'email': 'a.gonzalez@sesametime.com', 
   'company': 'Sesame', 
   'linkedin': 'https://www.linkedin.com/in/alba-gonz%C3%A1lez-vilches-23627670/', 
   'studyProgram': '', 
   'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Faa931eab-8b4f-4839-94c9-b327d4115e84%2F1738075692841.jpg/size/w=2000?exp=1752919535&sig=wggxo8aZvNYcEpoZw7nUaUNYg5AnLN-vHeD1vhjE0Hs&id=1decf8b7-c83c-80e3-bf3a-fd21454e58ff&table=block',
   'logo':"https://img.notionusercontent.com/ext/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fpublic.notion-static.com%2Ffffcffe1-09e9-488d-a8a4-996e5769475d%2FMTQyfGNvbnRlbnRzL2dHcVJYUmFRcHZuNlJqdXdVS09vLTE1MHgxNTAucG5n.webp/size/w=160?exp=1752834232&sig=OL89RrxZ9G14sLFmm-WFqfikAaODs5viJHt7w4Y793s&id=1decf8b7-c83c-805d-b4c6-007aced57a45&table=custom_emoji",
   
   }


trainer2 ={
    'name': 'Alfredo Merelo', 
    'position': 'Head of UX & Founder', 
    'email': 'alfredo@audaxstudio.com', 
    'company': 'Audax Studio', 
    'linkedin': 'https://es.linkedin.com/in/alfredomerelo', 
    'studyProgram': 'Sesión 10. UX para Startups', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Ff7cb65e1-de6d-41b3-b744-188ae57996af%2F1595330339019.jpg/size/w=50?exp=1752919659&sig=5aaAmDON_KK6mexXvBkXhYm7xySciMEi6VTnFnhHVag&id=1aecf8b7-c83c-81c4-925e-f64a33b62b2b&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F3095e86b-0931-4825-98a7-84aa7b0fe89b%2F4a1cb5203442427.5f50d441a35bd.png/size/w=250?exp=1752919762&sig=HBcyg0FO_mKMP8kiH1HW3CtodSXCTzU9cZ5rPU40jco&id=1aecf8b7-c83c-81c4-925e-f64a33b62b2b&table=block",
    
    }


trainer3 = { 
    'name': 'Antonio Perez', 
   'position': 'CIO', 
   'email': 'aperez@secture.com', 
   'company': 'Secture', 
   'linkedin': 'https://www.linkedin.com/in/antonioperezcano/', 
   'studyProgram': 'Sesión 6. Machine Learning y Automatización para Startups', 
   'photo': "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F35634f59-293b-43e7-b1cf-14f4652d8542%2F1565866228921.jpg/size/w=50?exp=1752919838&sig=7vXE_6UAMgrFY1CxbcGMcOxnfW5Jxj-JgUJSPbUEvLg&id=1bacf8b7-c83c-8070-8eed-d50034ecce45&table=block",
   "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F79442409-cb29-40e6-891d-f6f4af23830e%2F29481564.webp/size/w=250?exp=1752919834&sig=__cAyLLSkgD5cVuErxGHcbu3ZLcO3SElKQ5WYFvlTqw&id=1bacf8b7-c83c-8070-8eed-d50034ecce45&table=block",
   
   
   }


trainer4 = {
    'name': 'Carles Marcos i Guàrdia', 
    'position': 'Managing Director Spain & New Markets', 
    'email': 'carles.marcos@qonto.com', 
    'company': 'Qonto', 
    'linkedin': 'https://www.linkedin.com/in/carlesmarcosguardia/?originalSubdomain=fr', 
    'studyProgram': 'Sesión 14. Momentums Estratégico para Levantar Ronda de Inversión', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F11f7b994-eee5-48a5-a7e3-8a75e29a37a8%2FCarles_Marcos_i_Gurdia.jpeg/size/w=50?exp=1752920316&sig=O1b5ZAoT3E3a89Bpir6t25azJHtYd97J5yfWA1AzUV4&id=20ecf8b7-c83c-80af-afbe-c75309aaec57&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F8aaf3f18-04a9-48a7-b1a3-476e1f641d49%2Ficon.webp/size/w=250?exp=1752920312&sig=S6BElVw1jFr6vPFawZccNu6v2K7fteWZJ97gUvhiJAw&id=20ecf8b7-c83c-80af-afbe-c75309aaec57&table=block",
    
    
    }



trainer5 = {
    'name': 'Cristian Ull', 
    'position': 'Co-founder', 
    'email': 'cristian@101area.com', 
    'company': 'Area 101', 
    'linkedin': 'https://www.linkedin.com/in/cristianullmolina/', 
    'studyProgram': 'Sesión 13. Alianzas Corporativas', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fa9087c7c-1f96-4d43-81b7-8570a4d53d5f%2FWhatsApp-Image-2021-04-28-at-13.39.58.jpeg/size/w=50?exp=1752920382&sig=XeztyayamFCfLEZmSqFr3Gbpdh_hxK6C6rOKCun_7Vw&id=1aecf8b7-c83c-81d3-ba32-cae8bad49526&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Ffe97292f-ff1b-4244-8ac0-3e0a8b9ed5b0%2F101area_logo.jpg/size/w=250?exp=1752920379&sig=a_7m-4Glf6iXIhQRJKa7k6m4xZbFlTWds-_ALd1jr6E&id=1aecf8b7-c83c-81d3-ba32-cae8bad49526&table=block",
    
    
    }
    
trainer6 =  {
    'name': 'Cristina Aristoy', 
    'position': 'CEO', 
    'email': 'cristina@singularu.com', 
    'company': 'Singularu', 'photo': '', 
    'linkedin': 'https://www.linkedin.com/in/cristinaaristoycadenas/', 
    'studyProgram': 'Sesión 4. Marketing y ventas en B2C',
    "photo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F33d9e468-f601-4e33-9e13-a1f927f05b2c%2F74.jpg/size/w=50?exp=1752920451&sig=1-ydl5U35M8zoknt88NrGoQKkzlWePHAJ2VAL34CLC8&id=1aecf8b7-c83c-81a5-813e-c548d49dcc2b&table=block",
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F676164a6-3ccc-4e16-950a-e424df0e7770%2Fsingularu-ideas-a-medida-squarelogo-1617940231863.webp/size/w=250?exp=1752920445&sig=rUPqe2m1v0Wy3w5kl-xYP1nYvuSW9oeCOMnvFCvSxP4&id=1aecf8b7-c83c-81a5-813e-c548d49dcc2b&table=block",
    
    
    }




trainer7 ={
    'name': 'Cristina Lacasa', 
    'position': 'VP People & Culture', 
    'email': 'cristina.lacasa@saleslayer.com', 
    'company': 'Sales Layer', 
    'linkedin': 'https://www.linkedin.com/in/cristina-lacasa-moragon-99b55973/?originalSubdomain=es', 
    'studyProgram': 'Sesión 7. Escalabilidad Producto y Procesos', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F1c538182-756c-4b33-ac4a-772ca8f9ab57%2F64f7076458250d7a932a23a6_about_us_cris2x.png/size/w=50?exp=1752920527&sig=cfTOjlgxamp3m2cu83G24Khx9gHZvz_Dknn9_BTZpII&id=1c1cf8b7-c83c-8005-a93a-ebd3fbdca6ae&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fcf003182-d0c8-495f-b87b-565e8fb7b381%2Fsales_layer_logo.jpeg/size/w=250?exp=1752920523&sig=XwHRPB8jdggxTDy0keDIwrKYp6_cGBLHfD1kYpkHmio&id=1c1cf8b7-c83c-8005-a93a-ebd3fbdca6ae&table=block",
    
    
    }



trainer8 = {
    'name': 'Esther Blazquez', 
    'position': 'Coach Cultura & Liderazgo', 
    'email': 'esther@estherblazquez.com', 
    'company': 'Freelancer', 
    'linkedin': 'https://www.linkedin.com/in/esther-blazquez-blanco/?locale=es_ES', 
    'studyProgram': 'Sesión 11. Autogestión de Liderazgo', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F4f27dec1-a6e3-4685-a940-26bd7ee06771%2FNewdreamers-Cap.-8-Esther-Blazquez-post.png/size/w=50?exp=1752920592&sig=s4Y9vbKgPtLw2cK1k98W-QF9dS4ac9JBJkhzSl60zNk&id=1aecf8b7-c83c-81cf-b88d-ee198d833d14&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F50115822-f985-4cde-98c1-33be8ae49c14%2Festher_blazquez_blanco_page_logo.jpg/size/w=250?exp=1752920588&sig=1n64hyfJAN1P31rVc3NbODr2ITCpVzxuTtizi3qj7Rs&id=1aecf8b7-c83c-81cf-b88d-ee198d833d14&table=block",
    
    
    }



trainer9 =  {
    'name': 'Fran Benedito', 
    'position': 'Founder & CEO', 
    'email': 'fran@climatetrade.com', 
    'company': 'Climate Trade', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F193cef46-b7d7-48c1-9db2-a623a8f46038%2FFranB2_1.jpg/size/w=50?exp=1752920623&sig=oeeX7JqVqMzRd86KCozHTNxDqTi-K6zvWtod599VFEc&id=1c7cf8b7-c83c-80e2-af27-edfaf783eed6&table=block', 
    'linkedin': 'https://www.linkedin.com/in/fran-benedito/', 
    'studyProgram': '',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F77ed066a-03c5-4f52-9e6f-334a76d5bbe3%2Fclimatetrade_logo.jpeg/size/w=250?exp=1752920620&sig=qEDtwNsv1EKhl1Yw4aSHu1co6ia2vj07UdArsEYZyXA&id=1c7cf8b7-c83c-80e2-af27-edfaf783eed6&table=block",
    
    
    }


trainer10 = {
    'name': 'Gonzalo de Lomana', 
    'position': 'Co-Founder & CEO', 
    'email': 'gonzalo@saasgrowth.eu', 
    'company': 'SaaS Growth', 
    'linkedin': 'https://www.linkedin.com/in/gonzalodelomana/', 
    'studyProgram': 'Sesión 3. Escalabilidad en ventas B2B', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F813c6e9e-a1ff-417d-8e55-7dc47562f1cb%2F1627038155056.jpeg/size/w=50?exp=1752920751&sig=wt_06mtO09nb4H2GFPT2bBvXrTlon2N9WFYKNwAPXQo&id=1b5cf8b7-c83c-8097-ab4b-e8ff68bfc7ed&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F5b4baabe-5db5-4784-a109-bd36ca8ab19f%2Fsaas_growth_eu_logo.jpeg/size/w=250?exp=1752920747&sig=KuNEswqTxV7rd-su7d4OvTxKVKr-qQJMzpsr4diMiy0&id=1b5cf8b7-c83c-8097-ab4b-e8ff68bfc7ed&table=block",
    
    
    }



trainer11 = {
    'name': 'Iban Borràs', 
    'position': 'CPO', 
    'email': 'iban@saleslayer.com', 
    'company': 'Sales Layer', 
    'linkedin': 'https://www.linkedin.com/in/ibanborras/', 
    'studyProgram': 'Sesión 7. Escalabilidad Producto y Procesos', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F1d775094-bc31-49b4-a54a-87fb9b1c1b52%2F64f7077660905736014270e0_about_us_iban2x.png/size/w=50?exp=1752920813&sig=mGeBMyGNtqJaum-vSyGC55WqRAB-7PdaC7M1obxS8x8&id=1c1cf8b7-c83c-80e3-9eac-f5f2f221e6f2&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fff2983c9-fc87-42b2-87c6-f999c1895f90%2Fsales_layer_logo.jpeg/size/w=250?exp=1752920810&sig=OEjFL75uKX9HtkHhrPbR-7AF0WRPBign4nn6U_kuc8U&id=1c1cf8b7-c83c-80e3-9eac-f5f2f221e6f2&table=block",
    
    }



trainer12 = {
    'name': 'Ismael Vargas', 
    'position': 'Senior Growth Analyst', 
    'email': 'ismael@startupvalencia.org', 
    'company': 'Startup Valencia', 
    'linkedin': 'https://www.linkedin.com/in/ismael-vargas-soler/', 
    'studyProgram': 'Sesión 8. Evaluar resultados y rentabilidad en Marketing Digital', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F4d6dc32a-9443-4d9d-a96a-ce120a86854d%2F1643880231710.jpg/size/w=50?exp=1752920882&sig=W8RaSbzxiQiCTrR4ROlhSeEB7z9E4PrtfV90oN75ZF0&id=1decf8b7-c83c-80e6-94b2-c1c3f7f6ce8e&table=block',
    "logo": "https://img.notionusercontent.com/ext/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fpublic.notion-static.com%2Fdf8d34d2-bf1d-4cce-91c5-2eee6dd67eea%2Fimages.png/size/w=160?exp=1752835378&sig=EhrcIb7CWC77oLGMF4jY80tEaLOPYJxypWWpvSF5MDE&id=1decf8b7-c83c-80e2-b118-007a852ba9f9&table=custom_emoji",
    
    }



trainer13 = {
    'name': 'Javier Echaleku', 
    'position': 'Founder & CCO', 
    'email': 'echaleku@kuombo.com', 
    'company': 'Kuombo', 
    'linkedin': 'https://www.linkedin.com/in/echaleku/', 
    'studyProgram': 'Sesión 5. Embudos de Ventas', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F202e8b3b-d54c-4363-b519-2e465abe88ef%2F5104309.webp/size/w=50?exp=1752920948&sig=cTFvkF6lYG_6NoaEstWQpaB8Q6ZDmPR64ZtCx81aoGc&id=1aecf8b7-c83c-81ee-8482-cda6eeaaf1f4&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F8cf57d90-6ee5-4c88-9780-2c37329792bd%2FK-intro.svg/size/?exp=1752920944&sig=tsmczkY-KsWPOI2MDMbWbIm3j_Trv_MYejo3agXh_hA&id=1aecf8b7-c83c-81ee-8482-cda6eeaaf1f4&table=block",
    
    }




trainer14 = {
    'name': 'Javier Mora', 
    'position': 'Founder & CEO', 
    'email': 'javier@finded.io', 
    'company': 'Finded.io', 
    'linkedin': 'https://www.linkedin.com/in/javiermoraferrando/', 
    'studyProgram': 'Sesión 12. Gestión de Personas', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Ff6e7d32f-3727-4fc6-9350-18b6aef5db32%2F1703081103975.jpg/size/w=50?exp=1752921020&sig=td9GRGylmmBD4SpDbivysMhDLXSQjKtwUUnwMwyuAmY&id=1f9cf8b7-c83c-8007-9d71-df12e8107a43&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F871b6a00-cded-4575-8b8d-e2b363f36309%2Ffinded_io_logo.jpg/size/w=250?exp=1752921016&sig=hbc2iS9iDzpiRvGC1Mm4asSuRotG7xbHDQXYT6cdEsU&id=1f9cf8b7-c83c-8007-9d71-df12e8107a43&table=block",
    
    }


    
trainer15 = {
    'name': 'José Poyatos', 
    'position': 'PR & Communications Manager', 
    'email': 'jlpoyatos2006@hotmail.com', 
    'company': 'BigBuy', 
    'linkedin': 'https://www.linkedin.com/in/josepoyatos/', 
    'studyProgram': 'Sesión 2. Estrategia de Ventas, Elegir Modelos Tradicionales o Modelos Custom', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F658f7d92-9cea-4b9f-a0a4-2a0ee92950cc%2Fjpoyatos-bigbuy.jpg/size/w=50?exp=1752921087&sig=9_IKvJ6_nUjnzoVQ0PlJpp1qlkZ4NA6K4pQMfrBHVvc&id=1aecf8b7-c83c-8105-ad51-e17dc9329209&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F1775dd65-0677-4f2b-9ca7-960cc9efc276%2FLogotipo_BigBuy.png/size/w=250?exp=1752921084&sig=V8ajaqs8601I3E-hwZz3mX8UdTKyxtz-zBjPkUlGoE0&id=1aecf8b7-c83c-8105-ad51-e17dc9329209&table=block",
    
    }


trainer16 = {
    'name': 'Linda Janineh', 
    'position': 'Team Lead - Sales Development', 
    'email': 'linda@remote.com', 
    'company': 'Remote', 
    'linkedin': 'https://www.linkedin.com/in/lindajanineh/', 
    'studyProgram': 'Sesión 9. Internacionalización', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fbdd4ca84-116b-4388-b58e-53417ba7150f%2FScreenshot_2025-05-20_191550.png/size/w=50?exp=1752921142&sig=ixRZH61zmoYeSyQt5ipDTq9YApSGk-qv5Vs-LsYe_SM&id=1f9cf8b7-c83c-8056-a368-effd9e7285c4&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F1e9ad3e0-c3a5-41f4-96b0-657bbb707c0e%2Fremotecom_logo.jpg/size/w=250?exp=1752921138&sig=LrjEASVOG0hizHNQ7VELvE8092tTYqBu-iGspOk2cXA&id=1f9cf8b7-c83c-8056-a368-effd9e7285c4&table=block",
    
    }



trainer17 ={
    'name': 'Luca Zurlo', 
    'position': 'Advisor', 
    'email': 'lucazurlo@yahoo.co.uk', 
    'company': 'Gamaal', 
    'linkedin': 'https://www.linkedin.com/in/lucazurlo/', 
    'studyProgram': 'Sesión 15. Inversión', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fd0bb7bf4-432c-4739-a0bc-9565e34ca18f%2FLuca.jpg/size/w=50?exp=1752921198&sig=vevIFI6o6C6CUYLhO4CXIcaF0i4NyVN3RXqLmtj-Dyg&id=1b9cf8b7-c83c-8064-9b2b-e4a795fd6859&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F1240ada5-2008-49d8-a71b-b25b3cb547ba%2Fgamaal.jpg/size/w=250?exp=1752921195&sig=mFeTbSYNXfMsHiiyjCD7abPIaN0hGXDeVxGXRf2lPvU&id=1b9cf8b7-c83c-8064-9b2b-e4a795fd6859&table=block",
    }



trainer18 =  {
    'name': 'Majo Castillo', 
    'position': 'COO', 
    'email': '', 
    'company': 'Sesame', 
    'linkedin': 'https://www.linkedin.com/in/majocastillofauli/', 
    'studyProgram': 'Sesión 14. Momentums Estratégico para Levantar Ronda de Inversión', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F13ad94f9-d663-48c3-b4c2-87eceed415ed%2F1732191471166.jpg/size/w=50?exp=1752921260&sig=Q-rHS8rfNCpdf_ASREWcezTZbc7AZgoQYG1DFBcT1xg&id=1decf8b7-c83c-8037-905e-f6cb0962f677&table=block',
    "logo": "https://img.notionusercontent.com/ext/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fpublic.notion-static.com%2Ffffcffe1-09e9-488d-a8a4-996e5769475d%2FMTQyfGNvbnRlbnRzL2dHcVJYUmFRcHZuNlJqdXdVS09vLTE1MHgxNTAucG5n.webp/size/w=160?exp=1752835757&sig=DSbpwak-KuKzYeMAM-yD_nEGrW1MdjInNs3ErLEnEPE&id=1decf8b7-c83c-805d-b4c6-007aced57a45&table=custom_emoji",
    }

trainer19 = {
    'name': 'Pedro Redondo', 
    'position': 'CEO & Director de Negocio', 
    'email': 'pedro.redondo.gomez@gmail.com', 
    'company': 'Bilnea', 
    'linkedin': 'https://www.linkedin.com/in/pedroredondogomez/', 
    'studyProgram': 'Sesión 1. Desarrollo de Negocio y Nuevos Modelos de Crecimiento', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F6a6ef2f0-6326-4c41-90f3-6842cf1232ed%2FPEDRO-REDONDO-scaled.jpg/size/w=50?exp=1752921321&sig=UdOnEGc7eYqVK3KooN1Pvszg3xkxhudNa4CwWaZ6lxg&id=1aecf8b7-c83c-8183-b69e-c8be7f8b27cc&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F9684705f-5131-415f-960f-fb27af0e798c%2Fbilnea_logo.jpg/size/w=250?exp=1752921317&sig=jMlBFnFsjZMbOd7i1lN2GKmfWrRAtW8yoIGxY7WXwlc&id=1aecf8b7-c83c-8183-b69e-c8be7f8b27cc&table=block",
    
    }



trainer20 = {
    'name': 'Richard Morla', 
    'position': 'Founder', 
    'email': 'richardmorla@gmail.com', 
    'company': 'Rudo', 
    'linkedin': 'https://www.linkedin.com/in/richardmorla/', 
    'studyProgram': 'Sesión 14. Momentums Estratégico para Levantar Ronda de Inversión', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fbfaee4c3-e152-4040-ba9d-26b9475332a2%2F1730101227401.jpg/size/w=50?exp=1752921388&sig=nAzovYu2jXRcGzZeBW-h8N2QrzIsBr2dqwSDkQYsTLE&id=1decf8b7-c83c-808a-9024-ed6e41bfbe73&table=block',
    "logo": "https://img.notionusercontent.com/ext/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fpublic.notion-static.com%2Ff34fac7e-9cfd-411f-b001-408c1b91c43e%2Frudo_logo_(1).jpg/size/w=160?exp=1752835884&sig=73qomumLF3Q36AFhd-uSZfAd20U6L2ny4Q-V9KJkG98&id=1decf8b7-c83c-80e4-a8a6-007aa7c5cf64&table=custom_emoji",
    }



trainer21 =  {
    'name': 'Startup Valencia', 
    'position': 'Socios', 
    'email': '', 
    'company': 'Urban Sports, Zeus, Rudo, Vidext, ARTHEx Biotech', 
    'linkedin': '', 
    'studyProgram': 'Presentación del Programa. Dinámica Icebreaker Pitches y Networking Beers', 
    'photo': 'https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fbd962e3e-c554-4848-b308-dfc609160513%2F1080x1080_(1).png/size/w=250?exp=1752921705&sig=N-VbpDyL00djiZWjbGzHDnJt4CZv7ZTXUrfYRD8QLj4&id=1aecf8b7-c83c-81e9-bbeb-f5d842c605a0&table=block',
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fbd962e3e-c554-4848-b308-dfc609160513%2F1080x1080_(1).png/size/w=250?exp=1752921705&sig=N-VbpDyL00djiZWjbGzHDnJt4CZv7ZTXUrfYRD8QLj4&id=1aecf8b7-c83c-81e9-bbeb-f5d842c605a0&table=block",
    }



trainer22 = {
    'name': 'Toño Huerta', 
    'position': 'CEO', 
    'email': 'antonio@secture.com', 
    'company': 'Secture', 
    'linkedin': 'https://www.linkedin.com/in/tohuerta/', 
    'studyProgram': 'Sesión 6. Machine Learning y Automatización para Startups', 
    'photo': "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F90054b3d-5566-4d06-9e62-7fc01cbf45f9%2FCaptura_de_pantalla_2025-03-18_151327.png/size/w=50?exp=1752921484&sig=axsBck-K6kHKHOn68ulJ40Y1sBIK6t8AAkztFTWqSlY&id=1aecf8b7-c83c-81f1-9c9e-e36fd600248a&table=block",
    "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fc2ffbec9-906b-4b49-9040-5cb0bcd380bc%2F29481564.png/size/w=250?exp=1752921481&sig=fdswF7KC5cgUhM7d4n7mmLo82QW1o4_vTkLEAtuZGPA&id=1aecf8b7-c83c-81f1-9c9e-e36fd600248a&table=block",
    }





   
   

In [15]:
resultado = trainer.insert_many([trainer1,trainer2,trainer3,trainer4,trainer5,trainer6,trainer7,trainer8,trainer9,trainer10,
                                 trainer11,trainer12,trainer13,trainer14,trainer15,trainer16,trainer17,trainer18,trainer19,trainer20,trainer21,trainer22,
                                 ])
print(f"Usuario insertado con ID: {resultado.inserted_ids}")

Usuario insertado con ID: [ObjectId('687bb87795aa2580f6dec868'), ObjectId('687bb87795aa2580f6dec869'), ObjectId('687bb87795aa2580f6dec86a'), ObjectId('687bb87795aa2580f6dec86b'), ObjectId('687bb87795aa2580f6dec86c'), ObjectId('687bb87795aa2580f6dec86d'), ObjectId('687bb87795aa2580f6dec86e'), ObjectId('687bb87795aa2580f6dec86f'), ObjectId('687bb87795aa2580f6dec870'), ObjectId('687bb87795aa2580f6dec871'), ObjectId('687bb87795aa2580f6dec872'), ObjectId('687bb87795aa2580f6dec873'), ObjectId('687bb87795aa2580f6dec874'), ObjectId('687bb87795aa2580f6dec875'), ObjectId('687bb87795aa2580f6dec876'), ObjectId('687bb87795aa2580f6dec877'), ObjectId('687bb87795aa2580f6dec878'), ObjectId('687bb87795aa2580f6dec879'), ObjectId('687bb87795aa2580f6dec87a'), ObjectId('687bb87795aa2580f6dec87b'), ObjectId('687bb87795aa2580f6dec87c'), ObjectId('687bb87795aa2580f6dec87d')]


In [23]:
calendar = mydb["calendar"]

In [24]:
calendar1 = {
    "name": "Kickoff Meeting",
    "startDate": "2025-09-01",
    "startTime": "09:00",
    "endTime": "10:30",
    "location": "Main Conference Room"
}

calendar2 = {
    "name": "Product Launch",
    "startDate": "2025-10-15",
    "startTime": "14:00",
    "endTime": "16:00",
    "location": "Auditorium A"
}

calendar3 = {
    "name": "Networking Event",
    "startDate": "2025-11-20",
    "startTime": "18:30",
    "endTime": "21:00",
    "location": "Rooftop Terrace"
}

In [6]:
calendar_events = [calendar1, calendar2, calendar3]
processed_events = []

for event in calendar_events:
    start_date = event["startDate"]
    start_time = event["startTime"]
    end_time = event["endTime"]

    event["startDateTime"] = datetime.strptime(f"{start_date} {start_time}", "%Y-%m-%d %H:%M")
    print(f"'{event['name']}': startDateTime convertido a {event['startDateTime']}")

    event["endDateTime"] = datetime.strptime(f"{start_date} {end_time}", "%Y-%m-%d %H:%M")
    print(f"'{event['name']}': endDateTime convertido a {event['endDateTime']}")

    del event["startDate"]
    del event["startTime"]
    del event["endTime"]

    processed_events.append(event)

try:
    result = calendar.insert_many(processed_events)
    print(f"\n {len(result.inserted_ids)} eventos insertados en 'calendar'")
except Exception as e:
    print(f" Error al insertar en MongoDB: {e}")

NameError: name 'calendar1' is not defined

In [36]:
mentorship_collection = mydb["mentorship"]

In [37]:
mentorship1 = {
    "startDate": "2025-09-10",
    "time": "15:00",
    "mentor_id": "64f1a23b9cd0e3e1f10a1234",
    "startup_id": "64f1a23b9cd0e3e1f10a5678",
    "email": "contact@startupxyz.com",
    "participantsNames": "Ana López, Pedro García",
    "topics": "Business model validation, Marketing strategy",
    "observations": "Startup is in early stage, needs focus on customer acquisition",
    "suggestions": "Focus on social media campaigns and partnerships"
}

mentorship2 = {
    "startDate": "2025-10-05",
    "time": "11:00",
    "mentor_id": "64f1a23b9cd0e3e1f10a7890",
    "startup_id": "64f1a23b9cd0e3e1f10a2345",
    "email": "info@innovatehub.com",
    "participantsNames": "Sofía Pérez, Miguel Torres",
    "topics": "Product development, User feedback",
    "observations": "Product needs refinement in UX",
    "suggestions": "Conduct user testing and iterate design"
}

mentorship3 = {
    "startDate": "2025-11-20",
    "time": "09:30",
    "mentor_id": "64f1a23b9cd0e3e1f10a3456",
    "startup_id": "64f1a23b9cd0e3e1f10a6789",
    "email": "contact@startupabc.com",
    "participantsNames": "Laura Gómez, Carlos Ruiz",
    "topics": "Funding rounds, Investor pitch",
    "observations": "Startup is preparing for seed funding",
    "suggestions": "Refine pitch deck and financial projections"
}


In [ ]:
# Mentorships
mentorships = [mentorship1, mentorship2, mentorship3]

for m in mentorships:
    start_datetime_str = f"{m['startDate']} {m['time']}"
    m["startDateTime"] = datetime.strptime(start_datetime_str, "%Y-%m-%d %H:%M")
    print(f"'{m['mentor_id']}': startDateTime convertido a {m['startDateTime']}")
    # Borro los campos antiguos que ya no se usan
    del m["startDate"]
    del m["time"]

result = mentorship_collection.insert_many(mentorships)
print(f"\n{len(result.inserted_ids)} mentorías insertadas correctamente")


✅ '64f1a23b9cd0e3e1f10a1234': startDateTime convertido a 2025-09-10 15:00:00
✅ '64f1a23b9cd0e3e1f10a7890': startDateTime convertido a 2025-10-05 11:00:00
✅ '64f1a23b9cd0e3e1f10a3456': startDateTime convertido a 2025-11-20 09:30:00

🎉 3 mentorías insertadas correctamente


In [38]:
collection_startups = mydb["startup"]

In [39]:
startups = [
    {
        "company": "EDUCAYA",
        "description": "EDUCAYA es una empresa especializada en la innovación educativa, ofreciendo soluciones tecnológicas para el aprendizaje.",
        "sector": "Educación",
        "website": "educaya.es",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "",
        "contact": "ALICIA APARISI",
        "jobTitle": "co-founder",
        "email": "alicia.aparisi@gmail.com",
        "submissionTime": "March 20, 2025 4:39 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Faa42a7b0-64d1-4ce3-9bc5-6b97893f6e6e%2FCapturaTeTantallaT025-03-25T53018.png/size/w=50?exp=1752942220&sig=7k-M4-VlWd7NElnpXLadsojGnpUFo9EuahN0yxtmKnk&id=1bdcf8b7-c83c-8189-8ffe-edd3c58881aa&table=block",
        "hoursMentor": []
    },
    {
        "company": "Voicit",
        "description": "Voicit es la herramienta basada en IA para consultoras de RRHH que permite analizar entrevistas de forma rápida y eficiente.",
        "sector": "RRHH, IA, SaaS",
        "website": "voicit.es/",
        "stage": "Seed",
        "roundsRaised": "1",
        "awards": "Talent Summit, La Opinión",
        "contact": "Álvar",
        "jobTitle": "CEO",
        "email": "alvaroarres@voicit.com",
        "submissionTime": "March 20, 2025 5:19 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fe798da74-fcac-4a8c-bf1f-a0c6769b3909%2FCapturaTeTantallaT025-03-25T53245.png/size/w=50?exp=1752942258&sig=dT1d53P1Ke7LccUiTgJvxj-R-YZswZ41EOJ94NRDmA&id=1bccf8b7-c83c-81c8-8497-e9f1c770796a&table=block",
        "hoursMentor": []
    },
    {
        "company": "Kimera",
        "description": "Kimera es un SaaS B2B para la búsqueda y recomendación de productos eCommerce mediante tecnologías de IA.",
        "sector": "Software",
        "website": "kimeratechnologies.com/",
        "stage": "Seed",
        "roundsRaised": "1",
        "awards": "Mejor Iniciativa emprendedora (Premio Aitex 2022), CDTi Neotec (2023)",
        "contact": "Carles Igual",
        "jobTitle": "co-founder",
        "email": "carles@kimeratechnologies.com",
        "submissionTime": "March 20, 2025 4:39 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F0f93767a-2352-4dc9-b5ff-561f9c3774a0%2FCapturaTeTantallaT025-03-25T53323.png/size/w=50?exp=1752942523&sig=IJ5LTBVobk83ZZZrmg87EbLhznC1PuBTPI8GRzOT&id=1bccf8b7-c83c-8176-8baa-e73f7dfa8afc&table=block",
        "hoursMentor": []
    },
    {
        "company": "VENTUREST",
        "description": "Somos un Partner Digital. En Venturest ayudamos como un facilitador estratégico en la digitalización.",
        "sector": "Consultoría IT",
        "website": "venturest.app/",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "https://forinvest.feriavalencia.com/venturest-desarrolla-una-plataform…",
        "contact": "Javier Hernández",
        "jobTitle": "CEO",
        "email": "info@venturest.es",
        "submissionTime": "March 20, 2025 4:27 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F2e7a8fb3-5c92-4253-a2d3-517fe58ba49d%2FVENTU.jpg/size/w=50?exp=1752942592&sig=KCZJ4eQGlN9Fz66UzgUhiZWVitvkZ0T-eEMOotjIOT&id=1bccf8b7-c83c-8137-ac48-d85da61b1705&table=block",
        "hoursMentor": []
    },
    {
        "company": "ImpactE",
        "description": "Soluciones para la planificación, promoción y gestión de instalaciones de energía renovable.",
        "sector": "Energía",
        "website": "urbanimpacte.com",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "NEOTEC, mejor startup Govtech, más de 2M€ de ingreso acumulado entre proyectos.",
        "contact": "Carlos Prades Gil",
        "jobTitle": "consejero delegado",
        "email": "carlos.prades@urbanimpacte.com",
        "submissionTime": "March 20, 2025 12:32 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F24cfd771-322a-4963-ab99-d14e7ff81a82%2FCapturaTeTantallaT025-03-25T52805.png/size/w=50?exp=1752942629&sig=T-s7QwXMTT8IXpDjTtJBWs0r9iaKI14ATi7XoYdE&id=1bccf8b7-c83c-81fb-bea3-f200ae8cde2d&table=block",
        "hoursMentor": []
    },
    {
        "company": "MIOBIO",
        "description": "Transformamos datos médicos en menús personalizados listos para comer, optimizando salud y rendimiento.",
        "sector": "Deporte y alimentación personalizada",
        "website": "miobiosport.com/",
        "stage": "Seed",
        "roundsRaised": "1",
        "awards": "No",
        "contact": "Sergio",
        "jobTitle": "CEO",
        "email": "sergio@miobiosport.com",
        "submissionTime": "March 20, 2025 11:51 AM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fa92c4f91-c0da-416a-9eb3-4aeafdcf3be5%2FCapturaTeTantallaT025-03-25T54444.png/size/w=50?exp=1752942692&sig=PU0NIMXSNFShaG3O6ioYnIHgpuQXkbJvma3RVknWzLI&id=1bccf8b7-c83c-81d7-9f55-d53e546c1adc&table=block",
        "hoursMentor": []
    },
    {
        "company": "Clunnity",
        "description": "Software para clubes de fútbol.",
        "sector": "Sportstech",
        "website": "clunnity.com",
        "stage": "Pre-Seed",
        "roundsRaised": "1",
        "awards": "Finalista Valencia CF Innovation Hub",
        "contact": "Sergio Navarro",
        "jobTitle": "CEO",
        "email": "sergio@clunnity.com",
        "submissionTime": "March 18, 2025 6:44 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Ff657f354-a0fd-4765-b4f6-67adb8088557%2FCapturaTeTantallaT025-03-25T53956.png/size/w=50?exp=1752942988&sig=tHN8qhJRzYOA0SYf2rKUqc563YiNQZ2nef4wkm7u8t4&id=1bacf8b7-c83c-81a2-8e94-f6d8929c7c02&table=block",
        "hoursMentor": []
    },
    {
        "company": "ECKLETE COOP. V.",
        "description": "ECKLETE es una cooperativa de innovación que busca contribuir a alcanzar un futuro sostenible.",
        "sector": "Energía, arquitectura, planificación energética urbana, cooperación",
        "website": "ecklete.coop",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "I Premio en la categoría MathUp de la Misión Climática València 2030",
        "contact": "Edu Martín Castellanos",
        "jobTitle": "Presidente",
        "email": "administracion@ecklete.coop",
        "submissionTime": "March 20, 2025 10:46 AM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F18ae89a3-4635-4e84-b16b-12e6bb1bd8a7%2FCapturaTeTantallaT025-03-25T52647.png/size/w=50?exp=1752942783&sig=ytizNzw1CbnoYJpUWWsBkNQNuEjcwp3mJEn48VYueg4&id=1bccf8b7-c83c-8172-aa00-c30dd1b18234&table=block",
        "hoursMentor": []
    },
    {
        "company": "Verázial Labs",
        "description": "Desarrollamos e implantamos soluciones de identificación segura de personas y objetos.",
        "sector": "Salud / Seguridad",
        "website": "verazial.com/en/",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "Neotec 2023 / Icex Next 2024 / Mejor Startup rePCV 2024 / Top 100 Startups",
        "contact": "Alejandra Castillo",
        "jobTitle": "Digital Marketing Specialist",
        "email": "alejandra.castillo@verazial.com",
        "submissionTime": "March 20, 2025 10:13 AM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F707ff1ed-ec6d-4959-928e-316f7a44ddfc%2FCapturaTeTantallaT025-03-25T53705.png/size/w=50?exp=1752942841&sig=UZ80upI0C7iIZTI1h3dzUXiekuvH0s0iBhwi1jwFpM&id=1bccf8b7-c83c-818b-be15-de9f1f207773&table=block",
        "hoursMentor": []
    },
    {
        "company": "The Virtual Company",
        "description": "Desarrollo de soluciones 3D y de IA para el sector de Real Estate, V-commerce e industria.",
        "sector": "Proptech, healthcare, industrial",
        "website": "vcompany.co",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "Comunidad de Madrid, Lanzadera, Acelera Startups, Etiopía Zaragoza, Xtrartup",
        "contact": "Adrian Posada",
        "jobTitle": "CEO",
        "email": "Aposada@vcompany.co",
        "submissionTime": "March 19, 2025 12:44 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fd77ab2df-2c77-4977-ace9-7182af07c628%2FCapturaTeTantallaT025-03-25T53737.png/size/w=50?exp=1752942906&sig=8OkMwGPA66qz9IZGDnvJNfb393m7bQl8hGkCoC4n8ns&id=1bbcf8b7-c83c-8169-8652-dbf1f9dc6058&table=block",
        "hoursMentor": []
    },
    {
        "company": "Compartir Espacios (Kyoyu labs SL)",
        "description": "Compartir Espacios es una plataforma que conecta profesionales con espacios de trabajo flexibles.",
        "sector": "PropTech (Property Technology)",
        "website": "compartirespacios.com/",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "0",
        "contact": "Nacha Ruvira",
        "jobTitle": "CEO",
        "email": "info@compartirespacios.com",
        "submissionTime": "May 23, 2025 4:11 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F0a45c70f-4722-497a-8dbb-9bd1f644bd95%2FIMGT552.png/size/w=50?exp=1752943038&sig=ff6S5UWiCjWk3gsqmN22aEjGOMI0LP37P4itxFnoZWI&id=1fccf8b7-c83c-817f-9be1-e0d0aea28915&table=block",
        "hoursMentor": []
    },
    {
        "company": "Colibid",
        "description": "Colibid es una fintech española. Hemos desarrollado la primera plataforma de subastas de hipotecas, que permite a los clientes recibir ofertas hipotecarias personalizadas y precisas en minutos, mientras les ayuda a acceder fácilmente a los documentos requeridos por los bancos. Esto ahorra tiempo y dinero a los clientes. Ayudamos a los bancos a trabajar de manera más eficiente, proporcionando clientes de alta calidad y filtrados de manera completamente automatizada.También ofrecemos una tecnología basada en IA (Extracto.io) capaz de procesar documentos (sin importar su estructura o idioma) y convertirlos en datos en segundos, transfiriéndolos a cualquier sistema CRM o ERP a través de API.",
        "sector": "Fintech",
        "website": "colibid.com",
        "stage": "Seed",
        "roundsRaised": "Pre-seed y seed. €1,3m en total",
        "awards": "Top 10 Fin-Tech en South Summit 2022 - Top 10 Start-up en Premio El Referente 2022 - Ganador de la Mejor Innovación en el Global Forum Open Banking 2022 - Ganador de la startup más innovadora en Finnovati ng 2022 - Ganador de Top Startup 2022 en Finnovati ng 2022 - Top 3 de aplicaciones en E-Show Barcelona 2022 - Tercer Lugar Premio Emprende XXII CaixaBank 2023 (Comunidad Valenciana) - Las 10 mejores startups de tecnología fi nanciera en 4YFN 2024 en Barcelona - Tercer Lugar Premio Emprende XXI CaixaBank 2024 (Comunidad Valenciana) - Ganador Premio Emprende XXI CaixaBank 2024 (toda España)",
        "contact": "Jack O'Grady",
        "jobTitle": "COO",
        "email": "jack@colibid.com",
        "submissionTime": "April 23, 2025 5:10 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F13d86847-e24a-4d20-862f-1fc3b35e0199%2FCapturaTeTantallaT025-04-29T12526.png/size/w=50?exp=1752943123&sig=n3W72NF0UYLTyPiBqcv9eqHsMljmbMDvsU98NzVf8-Y&id=1decf8b7-c83c-811f-9e30-f58b67727a39&table=block",
        "hoursMentor": []
    },
    {
        "company": "Emendu",
        "description": "Emendu potencia a las pymes facilitando la incorporación y gestión tecnológica de forma ágil. Mediante su SaaS All-in-One, las pymes (entre 10 y 500 empleados) gestionan los dispositivos de su empresa. Renting de tecnología, IT support IA y gestión remota de dispositivos, todo en una plataforma integral que hace su vida más sencilla.",
        "sector": "SaaS B2B, Future of work, SMB focused",
        "website": "emendu.com/",
        "stage": "Seed",
        "roundsRaised": "2",
        "awards": "Selección Seed Rocket, Google for Startups, Decelera",
        "contact": "Aldebarán",
        "jobTitle": "Founder",
        "email": "abermell@emendu.com",
        "submissionTime": "April 9, 2025 4:36 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F0b9f0868-a96f-43c0-8a49-05850e96beee%2FCapturaTeTantallaT025-04-25T12952.png/size/w=50?exp=1752943193&sig=DHnPP0CnkyOgUXQEORGgoYoOXyq2Nh-CpeiurcHZRFc&id=1d0cf8b7-c83c-818a-9154-e44e77ccdeba&table=block",
        "hoursMentor": []
    },
    {
        "company": "Umibots",
        "description": "Umibots desarrolla soluciones de robótica autónoma en el ámbito de la limpieza urbana y de playas",
        "sector": "Robótica",
        "website": "umibots.eu/",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "Premio a la empresa innovación y turismo por ayuntamiento de Valencia",
        "contact": "Clodoaldo Gonzalez Fernández",
        "jobTitle": "CEO",
        "email": "clodo.gonzalez@umibots.eu",
        "submissionTime": "April 8, 2025 1:42 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Ffe03fa9e-964b-4175-832b-2987f5972dde%2FScreenshotT025-04-29T74525.png/size/w=50?exp=1752943239&sig=TRRxS-j68SKTvugSQh5vTuGAeSPY22VU05J87ArrGY&id=1cfcf8b7-c83c-815c-a0e8-c72be3547823&table=block",
        "hoursMentor": []
    },
    {
        "company": "Kenko Imalytics S.L.",
        "description": "Ayudamos al diagnóstico del cáncer que hacen los radiólogos a través de una plataforma que facilita la interpretación de las resonancias.",
        "sector": "medtech",
        "website": "biopsiavirtual.com/",
        "stage": "Pre-Seed",
        "roundsRaised": "2",
        "awards": "XXI Premio Consejo Social UPV 2022,   XVI Premio IDEAS UPV 2022,  Premio 5U-CV STARTUP 2024",
        "contact": "Gabriel Bestard Ribas",
        "jobTitle": "CEO",
        "email": "gabriel.bestard@kenkoimalytics.es",
        "submissionTime": "April 8, 2025 12:56 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fff6df31c-f428-4268-a2db-b1b7862a1415%2FScreenshotT025-04-29T74824.png/size/w=50?exp=1752943304&sig=U35gvxvVpcfslsrV6IewYWbyRVKBUcTATHNIsi793o&id=1cfcf8b7-c83c-8196-ba6c-cca9f4462fa2&table=block",
        "hoursMentor": []
    },
    {
        "company": "SimZero",
        "description": "Modelos virtuales científicos que replican la realidad en tiempo real con IA para la Industria 5.0 y ciudades inteligentes.",
        "sector": "Multisectorial",
        "website": "simzero.com",
        "stage": "Pre-Seed",
        "roundsRaised": "0",
        "awards": "HackerNoon Startups of The Year 2024",
        "contact": "Carlos Peña Monferrer",
        "jobTitle": "CEO & Co-Fundador",
        "email": "carlos@simzero.com",
        "submissionTime": "April 8, 2025 10:56 AM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Ff3c8d19c-5b07-41c7-828a-ca222f4b3edb%2FScreenshotT025-04-29T80411.png/size/w=50?exp=1752943345&sig=J54CYbfAKyTEELixFPx6g5WthrOCientGtArDf08vhk&id=1cfcf8b7-c83c-812c-b4b9-c58beadcaf04&table=block",
        "hoursMentor": []
    },
    {
        "company": "EBHealth3",
        "description": "Wellness Watcher es una experiencia de tranquilidad, seguridad, control y prevención en la salud digestiva de las personas",
        "sector": "Health Tech",
        "website": "wellnesswatcher.ai/",
        "stage": "Pre-Seed",
        "roundsRaised": "0",
        "awards": "Ganadores categoria Salud y Bienestar de los Premios a la innovación València Innovation Capital 2024.  Ganadores categoria Seed VIII Premios VLC Startup Awards. Ganadores XVII EDICIÓN PREMIOS IDEAS UPV 2023 categoría Lean",
        "contact": "Pablo Gonzalez Garcia",
        "jobTitle": "CEO",
        "email": "pg@ebhealth3.es",
        "submissionTime": "April 8, 2025 7:42 AM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F1073db04-8598-4274-a5ff-cf99ff0f1f3b%2FEBHealth3TOGOTinTondo.png/size/w=50?exp=1752943399&sig=ZCd5aA-hYl8Meh-AXKkcKE9MHSY18jvle9PZqQONNgU&id=1cfcf8b7-c83c-812d-ad2a-f2545c8e8790&table=block",
        "hoursMentor": []
    },
    {
        "company": "Fresh People",
        "description": "HR Saas B2B Software & Services",
        "sector": "hR",
        "website": "FreshPeople.team",
        "stage": "Seed",
        "roundsRaised": "1",
        "awards": "Best HR Startup Company in Europe",
        "contact": "Jose",
        "jobTitle": "CEO",
        "email": "Jose@freshpeople.team",
        "submissionTime": "April 8, 2025 1:10 AM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F5feb43f5-57e6-46d4-beb5-9f0ffc69c9ba%2FFreshPeopleTogotipoTzul.png/size/w=50?exp=1752943448&sig=qGcshzKnpGHBnzh1NTTdkmvR7HPWOFxmBZ5xYg8BTg&id=1cecf8b7-c83c-81d8-bd36-d0d2d96173c8&table=block",
        "hoursMentor": []
    },
    {
        "company": "Crowmie",
        "description": "Permitimos que empresas realicen la transición energética a coste 0 mientras que los inversores que participan en los proyectos que se instalan en estas reciben una rentabilidad del 9%",
        "sector": "Fintech Greentech",
        "website": "crowmie.com",
        "stage": "Seed",
        "roundsRaised": "2",
        "awards": "Forbes: Reconocmiento especial Startup, VSA: mejor startup seed 2024, The Gap In Between: Ganador en Financial Innovation for the common good, UNEF: Segundo premio Startup Solar, ElMundo & Caixabank: premio a las mejores 100 ideas de 2023, II Financial Adwards: Iniciativa más sostenible",
        "contact": "Pablo Valverde",
        "jobTitle": "CMO y cofounder",
        "email": "pablovalverde@crowmie.com",
        "submissionTime": "April 7, 2025 4:54 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fefb9faa5-b619-4996-ae4c-e555838c2ece%2FScreenshotT025-04-29T80219.png/size/w=50?exp=1752943501&sig=paG04GZAiWMx9ZsyWUSWSPL5doht-U1CuIOiL71CCes&id=1cecf8b7-c83c-8133-8e74-d5c37aeb541c&table=block",
        "hoursMentor": []
    },
    {
        "company": "Amaia Cuida",
        "description": "Amaia Cuida es la herramienta para personalizar los cuidados en centros sociosanitarios a través de la IA. La estandarización de los cuidados está quedando atrás gracias a la metodología de la Atención Centrada en la Persona (ACP), donde se sitúa a la persona en el centro a fin de cumplir con sus deseos y necesidades, es decir, personalizar sus cuidados. Esta metodología trae una nueva manera de trabajar y nuevas tareas. Pero, aquí nos encontramos con el principal pain de los centros sociosanitarios: el tiempo (malos ratios, rotación de personal, tareas administrativas, etc). Aquí es donde nace Amaia, la herramienta que ayuda a los centros sociosanitarios a realizar todas las tareas relacionadas con la ACP con un solo clic (Historias de Vida, preferencias, pictogramas, seguimientos). Y, lo más interesante es que nuestra IA analiza toda esta información de carácter psicosocial y la transforma en un plan de cuidados individualizado para cada una de las personas. ",
        "sector": "Salud",
        "website": "https://www.amaiacuida.com/",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "Finalistas Social Innovation Awards, ganadores Savia Innovación, finalistas Talento Jóven Levante, etc",
        "contact": "Lucía Villanueva",
        "jobTitle": "Co-founder y CMO",
        "email": "luciav@amaia-app.com",
        "submissionTime": "March 24, 2025 12:05 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Ff86129db-eb20-49ba-8461-d34292f9274b%2F1234567.png/size/w=50?exp=1752943672&sig=doE2VStJOC7qrP5DAWuPVA8gGnuMuZtK12E-ITyKjA&id=1c0cf8b7-c83c-81f6-a4d5-e4046465898b&table=block",
        "hoursMentor": []
    },
    {
        "company": "BATTERYFLY SMART ENERGY",
        "description": "Batteryfly es una compañía dedicada a la gestión de las baterías de movilidad tras su descarte en el uso del vehículo, o deficiencias en sus procesos productivos, dándole una segunda vida en entornos de movilidad y energía. Transformamos las baterías usadas para lograr nuevos productos que estén apoyados por energías sostenibles, y permitan el máximo aprovechamiento de los recursos ya empleados previamente.",
        "sector": "Energía y Movilidad",
        "website": "www.batteryfly.es",
        "stage": "Seed",
        "roundsRaised": "1 ronda de 500.000 €",
        "awards": "",
        "contact": "JAVIER ALCALÁ ESTERLICH",
        "jobTitle": "CEO",
        "email": "jalcala@batteryfly.es",
        "submissionTime": "April 7, 2025 3:21 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F58835205-334d-4fd5-ab15-f8022d77b77a%2F3.png/size/w=50?exp=1752943845&sig=7TlxJQz2nEX6LyFbdtvBxqDdZkYdZdOkLvTtTPvlis&id=1cecf8b7-c83c-81f7-bf45-dc258ddbe51f&table=block",
        "hoursMentor": []
    },
    {
        "company": "Mentelem - Mental health at work",
        "description": "Mentelem es un software de salud mental para empresas que evalúa, previene y acompaña el bienestar emocional en el trabajo.",
        "sector": "eHealth / HR Tech",
        "website": "mentelem.com/",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "Premio Nacional de Psicología 2023, Premio al emprendimiento social en…",
        "contact": "Alejandro Pereira",
        "jobTitle": "CEO y Founder",
        "email": "alejandro@mentelem.com",
        "submissionTime": "March 22, 2025 1:28 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F6344ca6f-7621-4b40-bf92-885f53445cd3%2FCapturaTeTantallaT025-03-25T54718.png/size/w=50?exp=1752944063&sig=wpid44GosgiPTWFjT7hIrr3Lk6FpRpBsmT99oFqEGI&id=1becf8b7-c83c-8168-b350-f8b1be7294d5&table=block",
        "hoursMentor": []
    },
    {
        "company": "GPTadvisor",
        "description": "GPTadvisor es una startup tecnológica, fundada en 2023, que aplica IA para asesoramiento financiero personalizado.",
        "sector": "Wealthtech",
        "website": "gptadvisor.com/",
        "stage": "Seed",
        "roundsRaised": 1,
        "awards": "N/A",
        "contact": "Isabel Forniés",
        "jobTitle": "COO & Co-founder",
        "email": "ifornies@gptadvisor.com",
        "submissionTime": "March 24, 2025 1:29 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2Fed82a92a-80e1-493a-b5c4-f33d6b44c741%2FLogosTPTadvisor.png/size/w=50?exp=1752944101&sig=gPyUMsk2mrzh6TuQWx87hjlGZt0JsnIFxopGByQfJI&id=1c0cf8b7-c83c-812b-888e-c527de78f262&table=block",
        "hoursMentor": []
    },
    {
        "company": "Your Friends Are Boring",
        "description": "Viajes en grupo para desconocidos que comparten intereses similares.",
        "sector": "TravelTech",
        "website": "yourfriendsareboring.com/",
        "stage": "Pre-Seed",
        "roundsRaised": 0,
        "awards": "Reconocidos por la lista Forbes 100 Most Creative People in Business; …",
        "contact": "Clara Haba",
        "jobTitle": "CEO",
        "email": "hello@yourfriendsareboring.com",
        "submissionTime": "March 24, 2025 4:23 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F1f60a39b-2d2c-4e20-83ac-5b492822e39a%2FCapturaTeTantallaT025-03-25T52517.png/size/w=50?exp=1752944140&sig=bAew7T7TVfbY1HIpT7HAUJ6oVvrXHqh4c1NaY79Gc&id=1c0cf8b7-c83c-8197-89e8-c09a69b554e3&table=block",
        "hoursMentor": []
    },
    {
        "company": "Feending",
        "description": "Un software inteligente para gestionar y dinamizar comunidades profesionales.",
        "sector": "Software",
        "website": "feending.com",
        "stage": "Seed",
        "roundsRaised": 1,
        "awards": "Ganadores del programa de aceleración de Wayra 2024. Ganadores del programa…",
        "contact": "Mario García-Granero",
        "jobTitle": "CEO",
        "email": "mario@feending.com",
        "submissionTime": "March 24, 2025 4:39 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F0bd5e40a-ec2e-432f-a72e-11b02a1a4ab5%2FCapturaTeTantallaT025-03-25T52405.png/size/w=50?exp=1752944175&sig=BjLv4h4hrSA3Phf2gxT60GbE05W68DTSfNzSLnSSU8&id=1c0cf8b7-c83c-81d8-a674-c16ef7c4beb3&table=block",
        "hoursMentor": []
    },
    {
        "company": "Celentis",
        "description": "Celentis es una empresa especializada en soluciones de detección y análisis químico, especialmente orientadas al ámbito de la seguridad y la salud pública. Se centra en el desarrollo de sistemas de detección de drogas ilícitas y otras sustancias mediante materiales híbridos y sensores químicos, apostando por productos de bajo coste, portátiles y de respuesta rápida.",
        "sector": "Salud y seguridad",
        "website": "https://celentis.bio/es/inicio/",
        "stage": "Seed",
        "roundsRaised": "0",
        "awards": "Premio 5UCV Junior 2023, Premio concurso de emprendimiento ideas de negocio UVEmpren 2023, , premio Spin UPV 2023, Premio mejor idea de negocio en fase avanzada UV 2024.",
        "contact": "Luis Antonio Morró González",
        "jobTitle": "CEO",
        "email": "lmorro@trl.plus",
        "submissionTime": "April 7, 2025 4:26 PM",
        "respondent": "Anónimo",
        "logo": "https://img.notionusercontent.com/s3/prod-files-secure%2F72ca6304-e33b-4a41-a30c-f2f25eed0ab9%2F6c76438c-58d7-478a-b0c1-919eb4b80e74%2FScreenshotT025-04-29T80952.png/size/w=50?exp=1752944650&sig=MGyvNTzUV6qq1D4ntWvJsjUC0cYTkszT8-CpxtjnE&id=1cecf8b7-c83c-8149-b1b6-c98588aea3d0&table=block",
        "hoursMentor": []
    }
]

new_mentor_data = [
    {'_id': ObjectId('687bc6a16b90add2cdd15aa9'), 'mentorName': 'Aktion Legal', 'hours': 0},
    {'_id': ObjectId('687bc6a16b90add2cdd15aa3'), 'mentorName': 'We Tangible', 'hours': 0},
    {'_id': ObjectId('687bc6a16b90add2cdd15aa7'), 'mentorName': 'VPL Tech', 'hours': 0},
    {'_id': ObjectId('687bc6a16b90add2cdd15aa4'), 'mentorName': 'Finded', 'hours': 0},
    {'_id': ObjectId('687bc6a16b90add2cdd15aa5'), 'mentorName': 'Audax Studio', 'hours': 0},
    {'_id': ObjectId('687bc6a16b90add2cdd15aa6'), 'mentorName': 'Lanáquera', 'hours': 0},
    {'_id': ObjectId('687bc6a16b90add2cdd15aa8'), 'mentorName': 'Innobide', 'hours': 0}
]

for startup in startups:
    startup["hoursMentor"] = new_mentor_data.copy()  # .copy() para evitar referencia compartida

print(startups[0]["hoursMentor"])


[{'_id': ObjectId('687bc6a16b90add2cdd15aa9'), 'mentorName': 'Aktion Legal', 'hours': 0}, {'_id': ObjectId('687bc6a16b90add2cdd15aa3'), 'mentorName': 'We Tangible', 'hours': 0}, {'_id': ObjectId('687bc6a16b90add2cdd15aa7'), 'mentorName': 'VPL Tech', 'hours': 0}, {'_id': ObjectId('687bc6a16b90add2cdd15aa4'), 'mentorName': 'Finded', 'hours': 0}, {'_id': ObjectId('687bc6a16b90add2cdd15aa5'), 'mentorName': 'Audax Studio', 'hours': 0}, {'_id': ObjectId('687bc6a16b90add2cdd15aa6'), 'mentorName': 'Lanáquera', 'hours': 0}, {'_id': ObjectId('687bc6a16b90add2cdd15aa8'), 'mentorName': 'Innobide', 'hours': 0}]


In [81]:
resultado = startup.insert_many([startup1,startup2,startup3])
print(f"Usuario insertado con ID: {resultado.inserted_ids}")

Usuario insertado con ID: [ObjectId('68792ce5df14b1b7d0017dbc'), ObjectId('68792ce5df14b1b7d0017dbd'), ObjectId('68792ce5df14b1b7d0017dbe')]


In [40]:
import locale
from dateutil import parser

for startup in startups:
    if isinstance(startup.get("submissionTime"), str):
        startup["submissionTime"] = parser.parse(startup["submissionTime"])

# --- Insertar todas las startups en MongoDB ---
try:
    if startups:  # Asegurarse de que la lista no esté vacía
        result = collection_startups.insert_many(startups)
        print(f"🎉 {len(result.inserted_ids)} startups insertadas correctamente ")
    else:
        print("ℹ️ No hay startups para insertar. La lista 'startups' está vacía.")
except Exception as e:
    print(f"❌ Error al insertar startups en MongoDB: {e}")

🎉 26 startups insertadas correctamente 


In [29]:
mentor_collection = mydb["mentor"]  

In [30]:
mentor1 = {
    "name": "Luis Fernández",
    "company": "TechMentors Inc.",
    "availabilityDate": "2025-08-15",
    "linkedin": "https://linkedin.com/in/luisfernandez",
    "logo": "https://example.com/photos/luis.png",
    "website": "https://techmentors.example.com",
    "position": "Senior Advisor",
    "mentoringAreas": "Startup Growth, Funding Strategies",
    "email": "luis@techmentors.example.com",
    "phone": "+34 600 654 321",
    "managementLink": "https://mentorships.example.com/luis"
}

mentor2 = {
    "name": "Ana Morales",
    "company": "InnovateHub",
    "availabilityDate": "2025-09-01",
    "linkedin": "https://linkedin.com/in/anamorales",
    "logo": "https://example.com/photos/ana.png",
    "website": "https://innovatehub.example.com",
    "position": "Business Coach",
    "mentoringAreas": "Marketing, Product Development",
    "email": "ana@innovatehub.example.com",
    "phone": "+1 415 555 1234",
    "managementLink": "https://mentorships.example.com/ana"
}

mentor3 = {
    "name": "John Smith",
    "company": "Startup Advisors",
    "availabilityDate": "2025-10-10",
    "linkedin": "https://linkedin.com/in/johnsmith",
    "logo": "https://example.com/photos/john.png",
    "website": "https://startupadvisors.example.com",
    "position": "Mentor",
    "mentoringAreas": "Investment, Scaling",
    "email": "john@startupadvisors.example.com",
    "phone": "+44 20 7946 0850",
    "managementLink": "https://mentorships.example.com/john",
    
}

In [31]:


mentors = [mentor1, mentor2, mentor3]

for m in mentors:
    m["availabilityDate"] = datetime.strptime(m["availabilityDate"], "%Y-%m-%d")
    print(f"'{m['name']}': fecha convertida a {m['availabilityDate']}")
    

result = mentor_collection.insert_many(mentors)
print(f"\n{len(result.inserted_ids)} mentores insertados correctamente")


'Luis Fernández': fecha convertida a 2025-08-15 00:00:00
'Ana Morales': fecha convertida a 2025-09-01 00:00:00
'John Smith': fecha convertida a 2025-10-10 00:00:00

3 mentores insertados correctamente


In [7]:
event_collection = mydb["event"]


In [27]:
event1 = {
    "name": "Annual Startup Summit",
    "location": "Madrid Conference Center",
    "startDate": "2025-10-10"
}

event2 = {
    "name": "Tech Innovation Fair",
    "location": "Barcelona Expo",
    "startDate": "2025-11-05"
}

event3 = {
    "name": "Investor Meetup",
    "location": "London Business Hub",
    "startDate": "2025-12-01"
}

In [28]:

events = [event1, event2, event3]

for e in events:
    e["startDateTime"] = datetime.strptime(f"{e['startDate']} 00:00", "%Y-%m-%d %H:%M")
    print(f"'{e['name']}': startDateTime convertido a {e['startDateTime']}")
    # Borro el campo antiguo startDate
    del e["startDate"]

result = event_collection.insert_many(events)
print(f"\n{len(result.inserted_ids)} eventos insertados correctamente")




'Annual Startup Summit': startDateTime convertido a 2025-10-10 00:00:00
'Tech Innovation Fair': startDateTime convertido a 2025-11-05 00:00:00
'Investor Meetup': startDateTime convertido a 2025-12-01 00:00:00

3 eventos insertados correctamente


In [20]:
import requests
import time

def get_coordinates(city_name):
    url = f"https://nominatim.openstreetmap.org/search"
    params = {
        "q": city_name,
        "format": "json",
        "limit": 1
    }
    headers = {"User-Agent": "event-extractor-bot"}
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        results = response.json()
        if results:
            lat = float(results[0]['lat'])
            lon = float(results[0]['lon'])
            return lon, lat
        else:
            print(f"⚠️ No se encontró coordenadas para '{city_name}'")
            return None
    except Exception as e:
        print(f"❌ Error obteniendo coordenadas de '{city_name}': {e}")
        return None

In [21]:
from datetime import datetime, timezone
from PIL import Image
from google import genai
import os
import json
import re

gemini_client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])

def generate_response(image_path):
    prompt = '''
You are analyzing an image that contains a table.

Your task:
- Extract each row of the table as a JSON object.
- Return the output as a JSON array (list of JSON objects).
- Use the following exact field names for each object:
    - "name": the name of the event
    - "location": the city where it takes place
    - "initialDate": the start date of the event (in the format "day month", e.g., "21 May")
    - "finalDate": the end date of the event (in the format "day month", e.g., "23 May")
    - "website": leave this field empty ("") if no website is visible in the image

✅ Important:
- Output only valid JSON.
- Do not explain anything, only return the JSON.
- Translate only the field names to English, leave the data as it appears (e.g., months in Spanish are OK).

✅ Example output:
[
    {
        "name": "Latitude 59",
        "location": "Tallinn",
        "initialDate": "21 Mayo",
        "finalDate": "23 Mayo",
        "website": ""
    }
]
'''
    image = Image.open(image_path)
    response = gemini_client.models.generate_content(
        model="gemini-2.5-pro",
        contents=[image, prompt]
    )
    return response.text


def convert_to_dict(json_file):
    json_string_clean = re.sub(r'^```json\n|\n```$', '', json_file.strip(), flags=re.MULTILINE)
    try:
        return json.loads(json_string_clean)
    except json.JSONDecodeError as e:
        print(f"ERROR parsing JSON: {e}")
        print(f"JSON content was:\n{json_string_clean}")
        return []  # devuelvo lista vacía si falla


def parse_date_safe(date_str):
    try:
        if not date_str:
            return None
        # Si solo viene '21 Junio', agregamos el año actual
        if any(mes in date_str.lower() for mes in ["enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]):
            if not any(char.isdigit() for char in date_str if char != ' '):  # proteges contra fechas absurdas
                return None
            if len(date_str.split()) == 2:
                current_year = datetime.now().year
                date_str += f" {current_year}"
        parsed_date = parser.parse(date_str, dayfirst=True)
        return parsed_date
    except Exception as e:
        print(f"⚠️ Warning: couldn't parse date '{date_str}', using None.")
        return None


def process_session(data):
    initial_date = parse_date(data.get("initialDate"))
    final_date = parse_date(data.get("finalDate"))

    location = data.get("location", "")
    coordinates = get_coordinates(location)

    location_point = {
        "type": "Point",
        "coordinates": coordinates
    } if coordinates else None

    session = {
        "name": data.get("name", "No Session Name"),
        "initialDate": initial_date.isoformat() if initial_date else None,
        "finalDate": final_date.isoformat() if final_date else None,
        "location": location,
        "website": data.get("website", ""),
        "location_point": location_point
    }
    return session


# Procesamiento principal
image_path = 'images/event/Captura de pantalla 2025-07-19 161539.png'
print(f"\n⏳ Procesando: {image_path}")

events_to_upload = []

try:
    gemini_output = generate_response(image_path)
    data_list = convert_to_dict(gemini_output)

    if isinstance(data_list, list):
        for data in data_list:
            session_doc = process_session(data)
            events_to_upload.append(session_doc)
            print(f"✅ Extraído sesión: {session_doc['name']}")
    else:
        print("❌ Error: la salida no es una lista")

except Exception as e:
    print(f"❌ Error procesando imagen: {e}")

print(f"\n🎯 Total sesiones extraídas: {len(events_to_upload)}")



⏳ Procesando: images/event/Captura de pantalla 2025-07-19 161539.png
✅ Extraído sesión: Latitude 59
✅ Extraído sesión: South Summit
✅ Extraído sesión: TNW
✅ Extraído sesión: Tech BBQ
✅ Extraído sesión: VDS
✅ Extraído sesión: Web Summit
✅ Extraído sesión: Slush

🎯 Total sesiones extraídas: 7


In [23]:
print(events_to_upload)

[{'name': 'Latitude 59', 'initialDate': '2025-07-19T15:50:47.554512+00:00', 'finalDate': '2025-07-19T15:50:47.554512+00:00', 'location': 'Tallinn', 'website': '', 'location_point': {'type': 'Point', 'coordinates': (24.7453688, 59.4372155)}, '_id': ObjectId('687bbf5995aa2580f6dec88e')}, {'name': 'South Summit', 'initialDate': '2025-07-19T15:50:47.836172+00:00', 'finalDate': '2025-07-19T15:50:47.836172+00:00', 'location': 'Madrid', 'website': '', 'location_point': {'type': 'Point', 'coordinates': (-3.7035825, 40.4167047)}, '_id': ObjectId('687bbf5995aa2580f6dec88f')}, {'name': 'TNW', 'initialDate': '2025-07-19T15:50:48.925967+00:00', 'finalDate': '2025-07-19T15:50:48.925967+00:00', 'location': 'Amsterdam', 'website': '', 'location_point': {'type': 'Point', 'coordinates': (4.8924534, 52.3730796)}, '_id': ObjectId('687bbf5995aa2580f6dec890')}, {'name': 'Tech BBQ', 'initialDate': '2025-07-19T15:50:49.846147+00:00', 'finalDate': '2025-07-19T15:50:49.846147+00:00', 'location': 'Copenhague', '

In [22]:
if events_to_upload:
    result = event_collection.insert_many(events_to_upload)
    print(f"\n🎉 {len(result.inserted_ids)} sesiones insertadas correctamente.")
else:
    print("\n⚠️ No se encontraron datos válidos para insertar.")


🎉 7 sesiones insertadas correctamente.


In [36]:
activity = mydb["activity"]

from datetime import datetime
import locale

In [37]:
activity1 = {
    'title': 'Investors Day', 
    'date': 'Diciembre 2025', 
    'schedule': 'TBD', 
    'location': 'Sala Multiusos, La Harinera', 
    'description': 'TBD', 
    'attendance': '', 
    'format': 'Presencial'}


activity2 =  {
    'title': 'Pitch Showcase', 
    'date': 'Julio 2025', 
    'schedule': 'TBD', 
    'location': 'Sala Multiusos, La Harinera', 
    'description': 'Mentores, formadores y agentes del ecosistema que se dividirán en comités de feedback para hacer sesiones de pitches simultáneas que puedan dar una primera valoración del pitch de cada startup.', 
    'attendance': 'Privado', 
    'format': ''}


activity3 = {
    'title': 'Reverse Pitches', 
    'date': 'Septiembre 2025', 
    'schedule': 'TBD', 
    'location': 'Sala Multiusos, La Harinera', 
    'description': 'En este evento, los participantes tendrán la oportunidad de escuchar presentaciones realizadas por los principales fondos de inversión y business angels. Durante las exposiciones, los inversores compartirán sus criterios clave para evaluar y seleccionar startups en las que invertir.', 
    'attendance': 'Privado', 
    'format': ''}

activity4 =  {
    'name': 'TBA', 
    'date': 'Noviembre 2025', 
    'schedule': 'TBD', 
    'location': 'Sala Multiusos, La Harinera', 
    'description': 'TBD', 
    'attendance': 'Privado', 
    'format': ''}



In [38]:

from datetime import datetime
import locale

# --- Configurar locale para español ---
try:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
except locale.Error:
    try:
        locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')
    except locale.Error:
        print("Advertencia: No se pudo establecer locale español.")

# --- Funciones ---
def parse_spanish_month_year(date_str):
    try:
        date_str_with_day = f"01 {date_str}"
        return datetime.strptime(date_str_with_day, "%d %B %Y")
    except ValueError:
        return None

def parse_spanish_month_year_manual(date_str):
    meses = {
        'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4,
        'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8,
        'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
    }
    try:
        mes, año = date_str.split()
        mes = mes.capitalize()
        return datetime(int(año), meses.get(mes, 1), 1)
    except Exception:
        return None

# --- Actividades ---
all_activities = [activity1, activity2, activity3, activity4]

# --- Procesamiento ---
processed_activities = []
for act in all_activities:
    doc = act.copy()

    if 'name' in doc and 'title' not in doc:
        doc['title'] = doc.pop('name')

    date_str = doc.get('date')
    if date_str:
        date_obj = parse_spanish_month_year(date_str) or parse_spanish_month_year_manual(date_str)
        if date_obj:
            doc['date'] = date_obj
            print(f"✅ '{doc['title']}': fecha '{date_str}' convertida a {date_obj}.")
        else:
            print(f"⚠️ '{doc['title']}': fecha '{date_str}' NO se pudo convertir, se mantiene string.")
    
    processed_activities.append(doc)

# Inserción correcta
result = activity.insert_many(processed_activities)



✅ 'Investors Day': fecha 'Diciembre 2025' convertida a 2025-12-01 00:00:00.
✅ 'Pitch Showcase': fecha 'Julio 2025' convertida a 2025-07-01 00:00:00.
✅ 'Reverse Pitches': fecha 'Septiembre 2025' convertida a 2025-09-01 00:00:00.
✅ 'TBA': fecha 'Noviembre 2025' convertida a 2025-11-01 00:00:00.
